In [ ]:
!cp -r /content/drive/MyDrive/Lenguaje/Proyecto/Modules Modules
!pip install top2vec

In [ ]:
from Modules.functions import get_labels, get_hierarchy_topic
from Modules.params import get_params
from top2vec import Top2Vec
from os.path import join
import hdbscan
import umap

In [ ]:
params = get_params()
filename = join(params["path models"],
                params["news model"])
model = Top2Vec.load(filename)

In [ ]:
umap_args = {
    "n_neighbors": 15,
    "n_components": 2,
    "metric": "cosine",
}
umap_model = umap.UMAP(**umap_args)
vectors = umap_model.fit(model.document_vectors)

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
cluster_model = hdbscan.HDBSCAN(min_cluster_size=15,
                                metric='euclidean',
                                cluster_selection_method='eom')
cluster = cluster_model.fit(umap_model.embedding_)
hierarchy_dict = get_hierarchy_topic(model,20)
clustered, outliers = get_labels(umap_model,
                                 model)
clustered["topic"] = clustered["doc_top"].apply(
    lambda label: hierarchy_dict[label])
filename = join(params["path results"],
                "news_clustered.csv")
clustered.to_csv(filename,
                 index=False)